In [ ]:
import pandas as pd
import re

df = pd.read_csv('/secure/shared_data/rag_tnm_results/summary/5_folds_summary/brca_df.csv')

data_lst = []
for i, row in df.iterrows():
    pathology_report = row["text"]
    ground_truth = row["t"]
    filename = row["patient_filename"]

    question_text = f"""
    Based on the following pathology report for a breast cancer patient, determine the pathologic T stage (T1, T2, T3, or T4) for breast cancer, according to the AJCC Cancer Staging Manual (7th edition). 
    Choose from T1, T2, T3, T4.

    {pathology_report}
    """
    data = {"Question": question_text, "Answer": ground_truth, "Filename": filename}
    data_lst.append(data)

,Unnamed: 0,patient_filename,t,text,type,n
0,46,TCGA-3C-AALI.84E6A935-1A49-4BC1-9669-3DEA161CF6FC,1,Path No.: Date Obtained: (Age: ). Date Receive...,BRCA,1
1,47,TCGA-3C-AALJ.265E5A9A-64FD-4B86-89BC-5E89F253C118,1,Path No.: Date Obtained: (Age: ). Date Receive...,BRCA,1
2,48,TCGA-3C-AALK.F43B01E6-E1DB-44B1-8003-93870606346A,0,Path No.: Date Obtained: (Age: ). Date Receive...,BRCA,-1
3,127,TCGA-4H-AAAK.8894688F-7167-48A1-BB1B-FC219B7675C2,1,Procedure: Left radical mastectomy. Preoperati...,BRCA,2
4,245,TCGA-5L-AAT0.F9B6971F-23C0-465F-BFEC-778BF228A1AE,1,Gender: Female. Color: White. Origin: Nature o...,BRCA,0
...,...,...,...,...,...,...
1026,1512,TCGA-WT-AB41.FDB00212-9782-4ACE-AD59-019384297A2F,1,SURG PATH #: SPECIMEN CLASS: ALT ID #: SEX: F....,BRCA,-1
1027,1513,TCGA-WT-AB44.B7EB0E0B-46C5-43C0-A78D-FB094290765A,0,SURG PATH #: SPECIMEN CLASS: ALT ID #: SEX: F....,BRCA,-1
1028,1514,TCGA-XX-A89A.5D85E578-64B4-4238-922E-802B8ED87800,2,Research Gross Description. Research Dx. Left ...,BRCA,0
1029,1515,TCGA-Z7-A8R5.7726F7AA-88A8-4DD6-B322-6FC68893E0D2,2,ADDENDUM. Addendum #1. Entered. BREAST CANCER ...,BRCA,1


In [4]:
import json
with open('/home/yl3427/cylab/SOAP_MA/Output/SOAP/sepsis_final_with_baseline.json', 'r') as file:
    data = json.load(file)
data[0]

{'Critical Care Specialist': {'expertise': ['Sepsis diagnosis',
   'Intensive care unit (ICU) management',
   'Mechanical ventilation',
   'Hemodynamic monitoring',
   'Vasopressor management'],
  'original_response': {'reasoning': 'To determine if the patient has sepsis, we need to evaluate the patient\'s clinical presentation, laboratory results, and other relevant information provided in the report. Sepsis is defined as a life-threatening organ dysfunction caused by a dysregulated host response to infection. The diagnosis of sepsis is typically based on the presence of a suspected or documented infection, along with signs of organ dysfunction, such as changes in mental status, cardiovascular instability, respiratory distress, or other signs of end-organ damage.\\n## Step 1: Evaluate the patient\'s clinical presentation for signs of infection or organ dysfunction."The patient has been on multiple antibiotics (vancomycin, piperacillin, levofloxacin, and piperacillin/tazobactam), which

In [3]:
df = pd.read_csv('/home/yl3427/cylab/SOAP_MA/data/mergedBioNLP2023.csv',
                        usecols=['File ID', 'Subjective', 'Objective', 'Assessment', 'Summary', 'cleaned_expanded_Summary', 'terms'])

df = df.fillna('').apply(lambda x: x.str.lower())
df['combined_summary'] = df['Summary'] + df['cleaned_expanded_Summary'] + df['terms']

mi = ["myocardial infarction", "elevation mi", "non-stemi", " NSTEMI", " stemi"]
chf = ["congestive heart failure", " chf", "HFrEF", "HFpEF"]
pulmonary_embolism = ["pulmonary embolism"]
pulmonary_hypertension = ["pulmonary hypertension", "pulmonary htn"]
sepsis = ["sepsis", "septic shock"]
urosepsis = ["urosepsis"]
meningitis = ["meningitis"]
aki = ["acute kidney injury", " aki", "acute renal failure", " arf"] # -> Acute tubular necrosis (ATN)인가 아닌가
atn = ["acute tubular necrosis", " atn"]
pancreatitis = ["pancreatitis"]
gi_bleed = ["gastrointestinal bleed", "gi bleed"]
hepatitis = ["hepatitis", " hep"]
cholangitis = ["cholangitis"]
asp_pneumonia = ["aspiration pneumonia"]

prob_dict = {'sepsis': sepsis, 
                'acute kidney injury': aki, 
                'pancreatitis': pancreatitis, 
                'gastrointestinal bleed': gi_bleed,
                "congestive heart failure": chf}

ids = set()
for name, lst in prob_dict.items():
    problem_terms = lst
    problem_terms = [term.lower() for term in problem_terms]

    # Use the first term as the primary term to check in the combined summary.
    primary_term = problem_terms[0]

    # Build a regex pattern that matches any of the problem terms.
    # pattern = '|'.join(problem_terms)
    pattern = '|'.join(re.escape(term) for term in problem_terms)

    mask = (
        df['combined_summary'].str.contains(pattern, na=False) &
        ~df['Subjective'].str.contains(pattern, na=False) &
        ~df['Objective'].str.contains(pattern, na=False)
    )

    filtered_df = df[mask]

    ids.update(filtered_df['File ID'])

df = df[df['File ID'].isin(ids)]
df.to_csv('/home/yl3427/cylab/SOAP_MA/Input/SOAP_5_problems.csv', index=False)
print(df.shape)

(351, 8)
